Data Cleaning and preparation completed

In [1]:
{
  "id": "global_skills_development_report",
  "name": "Global Skills Development Report",
  "url": "https://www.shl.com/products/product-catalog/view/global-skills-development-report/",
  "test_type": [
    "Aptitude",
    "Behavioral",
    "Cognitive",
    "Development",
    "English",
    "Personality & Behavior"
  ],
  "text_for_embedding": "Assessment name: Global Skills Development Report. Assessment types: Aptitude, Behavioral, Cognitive, Development, English, Personality & Behavior. Used for hiring and evaluating candidates based on skills, abilities, and behavior."
}


{'id': 'global_skills_development_report',
 'name': 'Global Skills Development Report',
 'url': 'https://www.shl.com/products/product-catalog/view/global-skills-development-report/',
 'test_type': ['Aptitude',
  'Behavioral',
  'Cognitive',
  'Development',
  'English',
  'Personality & Behavior'],
 'text_for_embedding': 'Assessment name: Global Skills Development Report. Assessment types: Aptitude, Behavioral, Cognitive, Development, English, Personality & Behavior. Used for hiring and evaluating candidates based on skills, abilities, and behavior.'}

In [1]:
import pandas as pd

In [2]:
data = pd.read_json('Raw_data.json')
df= pd.DataFrame(data)
df.head()
df["test_type"]
df

,name,url,remote_testing,adaptive_irt,test_type
0,Global Skills Development Report,https://www.shl.com/products/product-catalog/v...,False,False,"[A, E, B, C, D, P]"
1,.NET Framework 4.5,https://www.shl.com/products/product-catalog/v...,False,False,[K]
2,.NET MVC (New),https://www.shl.com/products/product-catalog/v...,False,False,[K]
3,.NET MVVM (New),https://www.shl.com/products/product-catalog/v...,False,False,[K]
4,.NET WCF (New),https://www.shl.com/products/product-catalog/v...,False,False,[K]
...,...,...,...,...,...
365,Written English v1,https://www.shl.com/products/product-catalog/v...,False,False,[K]
366,Written Spanish,https://www.shl.com/products/product-catalog/v...,False,False,[K]
367,Zabbix (New),https://www.shl.com/products/product-catalog/v...,False,False,[K]
368,360 Digital Report,https://www.shl.com/products/product-catalog/v...,False,False,[D]


In [3]:
TEST_TYPE_MAP = {
    "A": "Aptitude",
    "B": "Behavioral",
    "C": "Cognitive",
    "D": "Development",
    "E": "English",
    "F": "Functional",
    "K": "Knowledge & Skills",
    "P": "Personality & Behavior"
}



In [ ]:
TEST_TYPE_MAP = {
    "A": "Aptitude",
    "B": "Behavioral",
    "C": "Cognitive",
    "D": "Development",
    "E": "English",
    "F": "Functional",
    "K": "Knowledge & Skills",
    "P": "Personality & Behavior"
}

df["test_type"] = df["test_type"].apply(
    lambda codes: [TEST_TYPE_MAP[c] for c in codes if c in TEST_TYPE_MAP]
)


,name,url,remote_testing,adaptive_irt,test_type
0,Global Skills Development Report,https://www.shl.com/products/product-catalog/v...,False,False,[]
1,.NET Framework 4.5,https://www.shl.com/products/product-catalog/v...,False,False,[]
2,.NET MVC (New),https://www.shl.com/products/product-catalog/v...,False,False,[]
3,.NET MVVM (New),https://www.shl.com/products/product-catalog/v...,False,False,[]
4,.NET WCF (New),https://www.shl.com/products/product-catalog/v...,False,False,[]


In [6]:
def build_text_for_embedding(row):
    """
    Build a discriminative, retrieval-optimized text representation
    for embedding generation.
    """

    # --- Test types ---
    if isinstance(row.get("test_type"), list):
        test_types = ", ".join(row["test_type"])
    else:
        test_types = str(row.get("test_type", ""))

    # Normalize test type signal
    test_type_signal = test_types.lower()

    # --- Flags ---
    remote_signal = "remote testing" if row.get("remote_testing", False) else "on-site testing"
    adaptive_signal = (
        "adaptive testing using IRT"
        if row.get("adaptive_irt", False)
        else "standard fixed-form testing"
    )

    # --- Core role & skill intent (HIGH IMPACT) ---
    name = row["name"]

    role_intent = (
        f"This assessment evaluates job-relevant skills for {name} roles. "
        f"It focuses on role-specific knowledge, applied skills, and competency validation."
    )

    # --- Skill domain expansion ---
    if "knowledge" in test_type_signal or "skills" in test_type_signal:
        skill_focus = (
            "Assesses technical and functional skills required to perform job tasks, "
            "including domain knowledge, problem-solving, and practical application."
        )
    elif "personality" in test_type_signal or "behavior" in test_type_signal:
        skill_focus = (
            "Assesses behavioral traits, work style, personality characteristics, "
            "and interpersonal effectiveness relevant to job performance."
        )
    elif "cognitive" in test_type_signal:
        skill_focus = (
            "Assesses cognitive ability including logical reasoning, numerical reasoning, "
            "verbal reasoning, and learning agility."
        )
    else:
        skill_focus = (
            "Assesses general job-related competencies and workplace-relevant abilities."
        )

    # --- Usage context (important for query matching) ---
    usage_context = (
        "Used in hiring and pre-employment screening to evaluate candidate suitability, "
        "job readiness, and role fit."
    )

    return (
        f"Assessment: {name}. "
        f"Category: {test_types}. "
        f"{role_intent} "
        f"{skill_focus} "
        f"Testing mode: {remote_signal}. "
        f"Methodology: {adaptive_signal}. "
        f"{usage_context}"
    )


In [7]:
df["text_for_embedding"] = df.apply(build_text_for_embedding, axis=1)


In [8]:
df["text_for_embedding"].head(1)

0    Assessment: Global Skills Development Report. ...
Name: text_for_embedding, dtype: object

In [9]:
df["test_type"].apply(len).value_counts()


test_type
1    327
0     25
2     14
3      3
6      1
Name: count, dtype: int64

In [10]:
import re

def generate_id(name):
    name = name.lower()
    name = re.sub(r"[^a-z0-9\s]", "", name)
    return re.sub(r"\s+", "_", name)

df["id"] = df["name"].apply(generate_id)


In [11]:
df["id"] = df["name"].apply(generate_id)

df = df[["id"] + list(df.columns.drop("id"))]
len(df)


370

In [12]:
df.to_json("final_assessments.json", orient="records", indent=2)


In [13]:
data1 = {'query': 'I want to hire new graduates for a sales role in my company, the budget is for about an hour for each test. Give me some options', 'recommendations': [{'id': 'sales_interview_guide', 'name': 'Sales Interview Guide', 'url': 'https://www.shl.com/products/product-catalog/view/sales-interview-guide/', 'test_type': ['Personality & Behavior', 'Personality & Behavior']}, {'id': 'following_instructions_v1_us_r2', 'name': 'Following Instructions v1 - US (R2)', 'url': 'https://www.shl.com/products/product-catalog/view/following-instructions-v1-us-r2/', 'test_type': ['Knowledge & Skills']}, {'id': 'following_instructions_v1_uk_r1', 'name': 'Following Instructions v1 - UK (R1)', 'url': 'https://www.shl.com/products/product-catalog/view/following-instructions-v1-uk-r1/', 'test_type': ['Knowledge & Skills']}, {'id': 'entry_level_customer_servretail_contact_center', 'name': 'Entry Level Customer Serv-Retail & Contact Center', 'url': 'https://www.shl.com/products/product-catalog/view/entry-level-customer-serv-retail-and-contact-center/', 'test_type': ['Personality & Behavior', 'Cognitive']}, {'id': 'retail_sales_and_service_simulation', 'name': 'Retail Sales and Service Simulation', 'url': 'https://www.shl.com/products/product-catalog/view/retail-sales-and-service-simulation/', 'test_type': ['Behavioral', 'Knowledge & Skills', 'Aptitude']}, {'id': 'sales_transformation_report_10_sales_manager', 'name': 'Sales Transformation Report 1.0 - Sales Manager', 'url': 'https://www.shl.com/products/product-catalog/view/sales-transformation-report-sales-manager/', 'test_type': ['Personality & Behavior']}, {'id': 'sales_transformation_report_20_sales_manager', 'name': 'Sales Transformation Report 2.0 - Sales Manager', 'url': 'https://www.shl.com/products/product-catalog/view/sales-transformation-report-2-0-sales-manager/', 'test_type': ['Personality & Behavior']}, {'id': 'sales_transformation_10_individual_contributor', 'name': 'Sales Transformation 1.0 - Individual Contributor', 'url': 'https://www.shl.com/products/product-catalog/view/sales-transformation-report-individual-contributor/', 'test_type': ['Personality & Behavior']}, {'id': 'assessment_and_development_center_exercises', 'name': 'Assessment and Development Center Exercises', 'url': 'https://www.shl.com/products/product-catalog/view/assessment-and-development-center-exercises/', 'test_type': ['English']}, {'id': 'sales_transformation_20_individual_contributor', 'name': 'Sales Transformation 2.0 - Individual Contributor', 'url': 'https://www.shl.com/products/product-catalog/view/salestransformationreport2-0-individualcontributor/', 'test_type': ['Personality & Behavior']}]}


df2 = pd.DataFrame(data1['recommendations'])
df2

,id,name,url,test_type
0,sales_interview_guide,Sales Interview Guide,https://www.shl.com/products/product-catalog/v...,"[Personality & Behavior, Personality & Behavior]"
1,following_instructions_v1_us_r2,Following Instructions v1 - US (R2),https://www.shl.com/products/product-catalog/v...,[Knowledge & Skills]
2,following_instructions_v1_uk_r1,Following Instructions v1 - UK (R1),https://www.shl.com/products/product-catalog/v...,[Knowledge & Skills]
3,entry_level_customer_servretail_contact_center,Entry Level Customer Serv-Retail & Contact Center,https://www.shl.com/products/product-catalog/v...,"[Personality & Behavior, Cognitive]"
4,retail_sales_and_service_simulation,Retail Sales and Service Simulation,https://www.shl.com/products/product-catalog/v...,"[Behavioral, Knowledge & Skills, Aptitude]"
5,sales_transformation_report_10_sales_manager,Sales Transformation Report 1.0 - Sales Manager,https://www.shl.com/products/product-catalog/v...,[Personality & Behavior]
6,sales_transformation_report_20_sales_manager,Sales Transformation Report 2.0 - Sales Manager,https://www.shl.com/products/product-catalog/v...,[Personality & Behavior]
7,sales_transformation_10_individual_contributor,Sales Transformation 1.0 - Individual Contributor,https://www.shl.com/products/product-catalog/v...,[Personality & Behavior]
8,assessment_and_development_center_exercises,Assessment and Development Center Exercises,https://www.shl.com/products/product-catalog/v...,[English]
9,sales_transformation_20_individual_contributor,Sales Transformation 2.0 - Individual Contributor,https://www.shl.com/products/product-catalog/v...,[Personality & Behavior]
